In [ ]:
!pip install grpcio tensorflow-serving-api==2.14.1
!pip install git+https://github.com/antisoupbarrier/keras-image-helper.git

In [2]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [3]:
host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [4]:
from keras_image_helper import create_preprocessor

preprocessor = create_preprocessor('densenet', target_size=(150, 150))
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/Purple-Iris_pn.jpg/1200px-Purple-Iris_pn.jpg'
X = preprocessor.from_url(url)

In [ ]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = 'flower-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_2'].CopyFrom(np_to_protobuf(X))

In [13]:
pb_response = stub.Predict(pb_request, timeout=20.0)

preds = pb_response.outputs['dense_5'].float_val

In [11]:
classes = ['california_poppy',
 'iris',
 'magnolia',
 'bellflower',
 'astilbe',
 'water_lily',
 'dandelion',
 'sunflower',
 'calendula',
 'common_daisy',
 'tulip',
 'black_eyed_susan',
 'coreopsis',
 'carnation',
 'daffodil',
 'rose']

In [14]:
dict(zip(classes, preds))

{'california_poppy': 5.542286544368835e-06,
 'iris': 0.9989344477653503,
 'magnolia': 7.022139470791444e-06,
 'bellflower': 0.0009966477518901229,
 'astilbe': 1.5793312968526152e-06,
 'water_lily': 4.4488759158411995e-05,
 'dandelion': 3.195451370174851e-07,
 'sunflower': 3.99708397935683e-07,
 'calendula': 3.1259132811101153e-06,
 'common_daisy': 2.6688707066568895e-07,
 'tulip': 2.4865369141480187e-07,
 'black_eyed_susan': 1.3982562663272802e-08,
 'coreopsis': 7.071314644946369e-09,
 'carnation': 7.318598136407672e-07,
 'daffodil': 7.023081138868292e-07,
 'rose': 4.407945652928902e-06}